In [1]:
# from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
# from transformers import AutoTokenizer
# from torch.utils.data import DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import torch

/data/jieqi/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# from models.my_transformers import *
# from models.models import VAE, DDPM, MLPSkipNet, TransformerNet, VAE_DDPM
# from train.reconstruction import *
# from functions import weights_init_rondom
# from train import *

In [4]:
# class MyCollator(object):
#     def __init__(self, encoder_token, decoder_token):
#         self.encoder_token = encoder_token
#         self.decoder_token = decoder_token
#     def __call__(self, batch):
#         input_ids_bert = pad_sequence([torch.tensor(f['bert_token'], dtype=torch.long) for f in batch],
#                                   batch_first=True, padding_value=self.encoder_token)
#         input_ids_gpt = pad_sequence([torch.tensor(f['gpt2_token'], dtype=torch.long) for f in batch],
#                                     batch_first=True, padding_value=self.decoder_token)
#         try:
#             token_lengths = torch.tensor([[len(f['bert_token']), len(f['gpt2_token'])] for f in batch],
#                                         dtype=torch.long)
#         except:
#             token_lengths = torch.zeros((len(batch), 1091))
#             for i in range(len(batch)):
#                 token_lengths[i, len(batch[i]['gpt2_token'])] = 1
#         return (input_ids_bert, input_ids_gpt, token_lengths)
    
# encoder_model_class = MODEL_CLASS['BertForLatentConnectorAVG']
# tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
# latent_size = 64
# model_encoder = encoder_model_class.from_pretrained("prajjwal1/bert-small", latent_size=latent_size,
#                                                     pad_id=tokenizer_encoder.pad_token_id,local_files_only=False)


# decoder_model_class = MODEL_CLASS['GPT2ForLatentConnectorNew']
# tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")
# model_decoder = decoder_model_class.from_pretrained("gpt2-xl", latent_size=latent_size,
#                                                         latent_as_gpt_emb=True,
#                                                         latent_as_gpt_memory=True,local_files_only=False)
# decoder_n_layer = model_decoder.transformer.config.n_layer
# model_decoder.transformer.change_order()

# special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
# num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)
# model_decoder.resize_token_embeddings(len(tokenizer_decoder))
# bert_pad_token = tokenizer_encoder.pad_token_id
# gpt2_pad_token = tokenizer_decoder.pad_token_id

# my_collator = MyCollator(bert_pad_token, gpt2_pad_token)

# train_eval_dataset =load_dataset("guangyil/yelp_short_v2")
# eval_dataloader =  DataLoader(train_eval_dataset['test'], num_workers=0, collate_fn=my_collator,batch_size=64)
# train_dataloader = DataLoader(train_eval_dataset['train'], num_workers=0, collate_fn=my_collator, batch_size=64)

# output_dir = "test"
# model_vae = VAE(model_encoder, model_decoder, tokenizer_encoder, tokenizer_decoder, latent_size, output_dir)

# model_vae.apply(weights_init_rondom)
# model_vae.to('cuda')

# calc_rec_lgy(model_vae, tokenizer_encoder, tokenizer_decoder,eval_dataloader, "cuda", True, ns=1)

In [14]:
from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch


from models.my_transformers import *
from models.models import VAE, DDPM, MLPSkipNet, TransformerNet,VAE_DDPM
from train.reconstruction import *
from functions import weights_init_rondom
from train import *

In [17]:
class MyCollator(object):
    def __init__(self, encoder_token, decoder_token):
        self.encoder_token = encoder_token
        self.decoder_token = decoder_token
    def __call__(self, batch):
        input_ids_bert = pad_sequence([torch.tensor(f['bert_token'], dtype=torch.long) for f in batch],
                                  batch_first=True, padding_value=self.encoder_token)
        input_ids_gpt = pad_sequence([torch.tensor(f['gpt2_token'], dtype=torch.long) for f in batch],
                                    batch_first=True, padding_value=self.decoder_token)
        try:
            token_lengths = torch.tensor([[len(f['bert_token']), len(f['gpt2_token'])] for f in batch],
                                        dtype=torch.long)
        except:
            token_lengths = torch.zeros((len(batch), 1091))
            for i in range(len(batch)):
                token_lengths[i, len(batch[i]['gpt2_token'])] = 1
        return (input_ids_bert, input_ids_gpt, token_lengths)

def main():
    encoder_model_class = MODEL_CLASS['BertForLatentConnectorAVG']

    #initialize tokenizer and model
    print("initialize models")
    tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
    latent_size = 64
    model_encoder = encoder_model_class.from_pretrained("prajjwal1/bert-small", latent_size=latent_size,
                                                        pad_id=tokenizer_encoder.pad_token_id,local_files_only=False)


    decoder_model_class = MODEL_CLASS['GPT2ForLatentConnectorNew']
    tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")
    model_decoder = decoder_model_class.from_pretrained("gpt2-xl", latent_size=latent_size,
                                                            latent_as_gpt_emb=True,
                                                            latent_as_gpt_memory=True,local_files_only=False)
    decoder_n_layer = model_decoder.transformer.config.n_layer
    model_decoder.transformer.change_order()

    special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
    num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)
    model_decoder.resize_token_embeddings(len(tokenizer_decoder))
    bert_pad_token = tokenizer_encoder.pad_token_id
    gpt2_pad_token = tokenizer_decoder.pad_token_id

    my_collator = MyCollator(bert_pad_token, gpt2_pad_token)
    #download data
    print("download data")
    train_eval_dataset =load_dataset("guangyil/yelp_short_v2")
    eval_dataloader =  DataLoader(train_eval_dataset['test'], num_workers=0, collate_fn=my_collator,batch_size=8)
    train_dataloader = DataLoader(train_eval_dataset['train'], num_workers=0, collate_fn=my_collator, batch_size=8)

    output_dir = "test"
    model_vae = VAE(model_encoder, model_decoder, tokenizer_encoder, tokenizer_decoder, latent_size, output_dir)
    model_vae.apply(weights_init_rondom)
    model_vae.to('cuda:3')   
    ddpm = DDPM(eps_model=MLPSkipNet(latent_size), betas=(1e-4, 0.02), n_T=1000, criterion=nn.MSELoss(reduction='none'),).to('cuda:3')   
    ddpm.apply(weights_init_rondom)
    model = VAE_DDPM(model_vae, ddpm,1.0 ).to('cuda:3')   

    calc_ppl_lgy_ddpm(model.model_vae, tokenizer_encoder, tokenizer_decoder, ns=1,
                            ddpm=model.ddpm,
                            device='cuda:3'
                        )

print("here")


if __name__ == "__main__":
    main()

here
initialize models


Some weights of BertForLatentConnectorAVG were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['bert.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForLatentConnectorNew were not initialized from the model checkpoint at gpt2-xl and are newly initialized: ['h.48.attn.c_proj.bias', 'h.48.attn.c_attn.weight', 'lm_head.bias', 'h.48.ln_2.weight', 'h.48.attn.c_proj.weight', 'h.48.ln_1.bias', 'h.48.mlp.c_fc.bias', 'h.48.mlp.c_proj.bias', 'h.48.ln_1.weight', 'h.48.attn.c_attn.bias', 'linear_emb.weight', 'h.48.ln_2.bias', 'linear.weight', 'h.48.mlp.c_proj.weight', 'h.48.mlp.c_fc.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


download data



ÂÂÂ Dj Dj tack [* Windsor Windsor MV Bers Bers Bers Bers Bers Bers Bers Bers Bers Bers Bers Bers Sle Sle Sle Sle Sle Sle Sle Sle Pier Pier

admission admission admission admission admission admission admission admission admission admission che admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission admission

itaodanodanultureodanitaitaitaitaitaita sanctuaryitaitaitaitaitaitaitaitaitaitaitaitaitaitaitaitaitaitaitaita

Nova Pr Pr Pr Pr Pr Pr anim Pr anim anim anim anim anim anim anim second second second second second second second second second second second second second second second second

conf conf she she she she she scrape conf she she she she she Dresden Dresden Dresden Dresden Dresden she she she she she conf conf conf conf conf thrust she husband

Mit Mit Mit Mit IA Mit Ob Gov Ob Ob Ob Ob Ob Ob Ob Ob Ob Ob Ob Ob Ob Ob